<a href="https://colab.research.google.com/github/Jay-hv7/DL-Practice/blob/main/Torch_profiling_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision



In [2]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [00:00<00:00, 211MB/s] 


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [3]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try:
  urllib.URLopener().retrieve(url, filename)
except:
  urllib.request.urlretrieve(url, filename)

In [4]:
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename
                         )
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)

In [5]:
from torch.cuda import is_available
input_batch = input_tensor.unsqueeze(0)
print(torch.cuda.is_available())
if torch.cuda.is_available():
  input_batch = input_batch.to('cuda')
  model.to('cuda')

True


In [12]:
with torch.no_grad(), torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA]
) as prof:

  output = model(input_batch)
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([1.3789e-04, 1.3163e-04, 7.5630e-05, 1.1083e-04, 9.6404e-05, 1.3945e-04,
        2.1075e-04, 1.2519e-04, 1.0905e-04, 3.8484e-05, 9.4773e-05, 6.9633e-05,
        1.0472e-04, 1.2593e-04, 2.7572e-04, 1.1175e-04, 7.3579e-05, 1.1755e-04,
        1.5517e-04, 1.3978e-04, 1.8342e-04, 8.0105e-05, 1.3487e-04, 6.9928e-05,
        1.1782e-04, 9.3183e-05, 9.3396e-05, 1.1701e-04, 1.1987e-04, 1.0378e-04,
        1.4469e-04, 1.1704e-04, 4.3569e-04, 9.3473e-05, 3.1708e-04, 5.4145e-05,
        1.9219e-04, 7.7250e-05, 9.0401e-05, 1.0144e-04, 1.6749e-04, 9.8975e-05,
        1.3058e-04, 1.7978e-04, 1.0211e-04, 1.1947e-04, 7.4048e-05, 1.6887e-04,
        1.3720e-04, 1.4427e-04, 1.0270e-04, 1.3691e-04, 1.5659e-04, 8.1339e-05,
        3.1305e-04, 2.6110e-04, 5.6214e-05, 1.0892e-04, 1.0703e-04, 1.3314e-04,
        1.0379e-04, 1.7033e-04, 7.4884e-05, 5.5784e-05, 1.0887e-04, 1.4640e-04,
        1.5442e-04, 3.2324e-04, 1.7711e-04, 9.7884e-05, 1.0745e-04, 1.0405e-04,
        4.1513e-05, 7.8393e-05, 1.4158e-

In [13]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2025-12-04 09:31:48--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.1’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2025-12-04 09:31:48 (109 MB/s) - ‘imagenet_classes.txt.1’ saved [10472/10472]



In [14]:
with open('imagenet_classes.txt',"r") as f:
  categories = [s.strip() for s in f.readlines()]
print(categories)

['tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead', 'electric ray', 'stingray', 'cock', 'hen', 'ostrich', 'brambling', 'goldfinch', 'house finch', 'junco', 'indigo bunting', 'robin', 'bulbul', 'jay', 'magpie', 'chickadee', 'water ouzel', 'kite', 'bald eagle', 'vulture', 'great grey owl', 'European fire salamander', 'common newt', 'eft', 'spotted salamander', 'axolotl', 'bullfrog', 'tree frog', 'tailed frog', 'loggerhead', 'leatherback turtle', 'mud turtle', 'terrapin', 'box turtle', 'banded gecko', 'common iguana', 'American chameleon', 'whiptail', 'agama', 'frilled lizard', 'alligator lizard', 'Gila monster', 'green lizard', 'African chameleon', 'Komodo dragon', 'African crocodile', 'American alligator', 'triceratops', 'thunder snake', 'ringneck snake', 'hognose snake', 'green snake', 'king snake', 'garter snake', 'water snake', 'vine snake', 'night snake', 'boa constrictor', 'rock python', 'Indian cobra', 'green mamba', 'sea snake', 'horned viper', 'diamondback', 

In [15]:
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
  category = categories[top5_catid[i]]
  probability = top5_prob[i].item()
  print(f"category: {category}, Probability {probability:.4f}")

category: Samoyed, Probability 0.8238
category: Arctic fox, Probability 0.0142
category: white wolf, Probability 0.0123
category: Pomeranian, Probability 0.0079
category: keeshond, Probability 0.0064


In [16]:
print(prof.key_averages().table(sort_by='cuda_time_total'))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv2d         1.36%     232.746us        43.78%       7.468ms      79.442us       0.000us         0.00%      10.881ms     115.760us            94  
                                      aten::convolution         2.42%     412.693us        42.42%       7.235ms      76.966us       0.000us         0.00%      10.881ms     115.760us            94  
         